<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Advent-of-code-2019" data-toc-modified-id="Advent-of-code-2019-1">Advent of code 2019</a></span><ul class="toc-item"><li><span><a href="#Day-8" data-toc-modified-id="Day-8-1.1">Day 8</a></span><ul class="toc-item"><li><span><a href="#Part-1" data-toc-modified-id="Part-1-1.1.1">Part 1</a></span></li><li><span><a href="#Part-2" data-toc-modified-id="Part-2-1.1.2">Part 2</a></span></li></ul></li></ul></li></ul></div>

# Advent of code 2019

https://adventofcode.com/2019

## Day 8

In [3]:
def get_opcode(value):
    opcode = value % 100
    if opcode not in (1, 2, 3, 4, 5, 6, 7, 8, 99):
        raise ValueError("ERROR: {} is wrong opcode".format(opcode))
    
    list_modes = []
    modes = value // 100
    for i in range(3):
        temp_mode, modes = modes % 10, modes // 10
        list_modes.append(temp_mode)

    if any(True if u not in (0, 1) else False for u in list_modes):
        raise ValueError("ERROR: wrong parameter code in {}".format(value))
    
    return opcode, list_modes

def get_param(values, pos, param_num, modes):
    val = values[pos+param_num]
    if modes[param_num-1] == 0:
        return values[val]
    elif modes[param_num-1] == 1:
        return val
    elif modes[param_num-1] == 2:
        return None
    else return None

def run_intcode(array):
    values = array[:]

    i = 0
    N = len(values)
    while i < N:
        opcode, modes = get_opcode(values[i])
        # print(opcode, modes)

        if opcode == 99:
            yield None
        elif opcode == 1:
            if modes[2] == 0:
                val1 = get_param(values, i, 1, modes)
                val2 = get_param(values, i, 2, modes)
                values[values[i+3]] = val1 + val2
            i += 4
        elif opcode == 2:
            if modes[2] == 0:
                val1 = get_param(values, i, 1, modes)
                val2 = get_param(values, i, 2, modes)
                values[values[i+3]] = val1 * val2
            i += 4
        elif opcode == 3:
            if modes[0] == 0:
                values[values[i+1]] = (yield)
            i += 2
        elif opcode == 4:
            val1 = get_param(values, i, 1, modes)
            yield val1
            i += 2
        elif opcode == 5:
            val1 = get_param(values, i, 1, modes)
            val2 = get_param(values, i, 2, modes)
            if val1 != 0:
                i = val2
            else:
                i += 3
        elif opcode == 6:
            val1 = get_param(values, i, 1, modes)
            val2 = get_param(values, i, 2, modes)
            if val1 == 0:
                i = val2
            else:
                i += 3
        elif opcode == 7:
            val1 = get_param(values, i, 1, modes)
            val2 = get_param(values, i, 2, modes)
            if modes[2] == 0:
                if val1 < val2:
                    values[values[i+3]] = 1
                else:
                    values[values[i+3]] = 0
            i += 4
        elif opcode == 8:
            val1 = get_param(values, i, 1, modes)
            val2 = get_param(values, i, 2, modes)
            if modes[2] == 0:
                if val1 == val2:
                    values[values[i+3]] = 1
                else:
                    values[values[i+3]] = 0
            i += 4

    return

### Part 1

In [4]:
def test_max_thruster(f):
    assert f([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]) == 43210
    assert f([3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]) == 54321
    assert f([
        3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0
    ]) == 65210

In [65]:
def calc_thruster(intcode, sequence):
    thrust = 0
    for s in sequence:
        output = run_intcode(intcode, input_nums=[s, thrust])
        thrust = output[0]
    return thrust

In [5]:
intcode = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]

In [31]:
def calc_thruster(intcode, sequence):
    amps = []
    for i in range(5):
        amps.append(run_intcode(intcode))
        amps[i].send(None)
        amps[i].send(sequence[i])

    thrust = 0
    for i in range(5):
        thrust = amps[i].send(thrust)

    return thrust

In [32]:
from itertools import permutations

def get_max_thrust(intcode):
    seqs = permutations(range(5))

    return max(calc_thruster(intcode, sequence) for sequence in seqs)
    
#     max_thrust = 0
#     max_thrust_sequence = None
#     for sequence in seqs:
#         thrust = calc_thruster(intcode, sequence)
#         if thrust > max_thrust:
#             max_thrust = thrust
#             max_thrust_sequence = sequence
#     print(max_thrust_sequence)
#     return max_thrust

In [33]:
test_max_thruster(get_max_thrust)

In [36]:
input_file = 'input/input_day09.txt'
with open(input_file, 'r') as fin:
    data = fin.readline().strip()
    data = [int(u) for u in data.split(',')]

### Part 2